In [4]:
import joblib
from egbz.utils import *
# ^^^ pyforest auto-imports - don't write above this line


In [6]:
data = joblib.load('../training_data/data.dataset')

In [33]:
ts = from_df(data.sel(zone='75').to_dataframe())

In [50]:
from darts.models.block_rnn_model import *
from darts.dataprocessing.transformers.scaler import Scaler

In [52]:
scaler = Scaler()
ts = scaler.fit_transform(ts)

In [53]:
model = BlockRNNModel(model='GRU', input_chunk_length=30, output_chunk_length=1)

In [62]:
history = model.historical_forecasts(ts, last_points_only=False)

[2021-08-30 22:05:09,048] INFO | darts.models.torch_forecasting_model | Train dataset contains 126 samples.
[2021-08-30 22:05:09,048] INFO | darts.models.torch_forecasting_model | Train dataset contains 126 samples.
[2021-08-30 22:05:13,320] INFO | darts.models.torch_forecasting_model | Train dataset contains 127 samples.
[2021-08-30 22:05:13,320] INFO | darts.models.torch_forecasting_model | Train dataset contains 127 samples.
[2021-08-30 22:05:17,361] INFO | darts.models.torch_forecasting_model | Train dataset contains 128 samples.
[2021-08-30 22:05:17,361] INFO | darts.models.torch_forecasting_model | Train dataset contains 128 samples.
[2021-08-30 22:05:21,513] INFO | darts.models.torch_forecasting_model | Train dataset contains 129 samples.
[2021-08-30 22:05:21,513] INFO | darts.models.torch_forecasting_model | Train dataset contains 129 samples.
[2021-08-30 22:05:25,947] INFO | darts.models.torch_forecasting_model | Train dataset contains 130 samples.
[2021-08-30 22:05:25,947] IN

[2021-08-30 22:08:09,129] INFO | darts.models.torch_forecasting_model | Train dataset contains 164 samples.
[2021-08-30 22:08:09,129] INFO | darts.models.torch_forecasting_model | Train dataset contains 164 samples.
[2021-08-30 22:08:14,485] INFO | darts.models.torch_forecasting_model | Train dataset contains 165 samples.
[2021-08-30 22:08:14,485] INFO | darts.models.torch_forecasting_model | Train dataset contains 165 samples.
[2021-08-30 22:08:19,860] INFO | darts.models.torch_forecasting_model | Train dataset contains 166 samples.
[2021-08-30 22:08:19,860] INFO | darts.models.torch_forecasting_model | Train dataset contains 166 samples.
[2021-08-30 22:08:25,164] INFO | darts.models.torch_forecasting_model | Train dataset contains 167 samples.
[2021-08-30 22:08:25,164] INFO | darts.models.torch_forecasting_model | Train dataset contains 167 samples.
[2021-08-30 22:08:30,467] INFO | darts.models.torch_forecasting_model | Train dataset contains 168 samples.
[2021-08-30 22:08:30,467] IN

[2021-08-30 22:12:08,824] INFO | darts.models.torch_forecasting_model | Train dataset contains 202 samples.
[2021-08-30 22:12:08,824] INFO | darts.models.torch_forecasting_model | Train dataset contains 202 samples.
[2021-08-30 22:12:16,282] INFO | darts.models.torch_forecasting_model | Train dataset contains 203 samples.
[2021-08-30 22:12:16,282] INFO | darts.models.torch_forecasting_model | Train dataset contains 203 samples.
[2021-08-30 22:12:23,985] INFO | darts.models.torch_forecasting_model | Train dataset contains 204 samples.
[2021-08-30 22:12:23,985] INFO | darts.models.torch_forecasting_model | Train dataset contains 204 samples.
[2021-08-30 22:12:31,663] INFO | darts.models.torch_forecasting_model | Train dataset contains 205 samples.
[2021-08-30 22:12:31,663] INFO | darts.models.torch_forecasting_model | Train dataset contains 205 samples.
[2021-08-30 22:12:39,188] INFO | darts.models.torch_forecasting_model | Train dataset contains 206 samples.
[2021-08-30 22:12:39,188] IN

[2021-08-30 22:17:25,426] INFO | darts.models.torch_forecasting_model | Train dataset contains 240 samples.
[2021-08-30 22:17:25,426] INFO | darts.models.torch_forecasting_model | Train dataset contains 240 samples.
[2021-08-30 22:17:34,178] INFO | darts.models.torch_forecasting_model | Train dataset contains 241 samples.
[2021-08-30 22:17:34,178] INFO | darts.models.torch_forecasting_model | Train dataset contains 241 samples.
[2021-08-30 22:17:43,051] INFO | darts.models.torch_forecasting_model | Train dataset contains 242 samples.
[2021-08-30 22:17:43,051] INFO | darts.models.torch_forecasting_model | Train dataset contains 242 samples.
[2021-08-30 22:17:51,981] INFO | darts.models.torch_forecasting_model | Train dataset contains 243 samples.
[2021-08-30 22:17:51,981] INFO | darts.models.torch_forecasting_model | Train dataset contains 243 samples.
[2021-08-30 22:18:01,136] INFO | darts.models.torch_forecasting_model | Train dataset contains 244 samples.
[2021-08-30 22:18:01,136] IN

[2021-08-30 22:23:24,473] INFO | darts.models.torch_forecasting_model | Train dataset contains 278 samples.
[2021-08-30 22:23:24,473] INFO | darts.models.torch_forecasting_model | Train dataset contains 278 samples.
[2021-08-30 22:23:34,558] INFO | darts.models.torch_forecasting_model | Train dataset contains 279 samples.
[2021-08-30 22:23:34,558] INFO | darts.models.torch_forecasting_model | Train dataset contains 279 samples.
[2021-08-30 22:23:44,595] INFO | darts.models.torch_forecasting_model | Train dataset contains 280 samples.
[2021-08-30 22:23:44,595] INFO | darts.models.torch_forecasting_model | Train dataset contains 280 samples.
[2021-08-30 22:23:54,629] INFO | darts.models.torch_forecasting_model | Train dataset contains 281 samples.
[2021-08-30 22:23:54,629] INFO | darts.models.torch_forecasting_model | Train dataset contains 281 samples.
[2021-08-30 22:24:04,792] INFO | darts.models.torch_forecasting_model | Train dataset contains 282 samples.
[2021-08-30 22:24:04,792] IN

In [63]:
history

[<TimeSeries (DataArray) (time: 1, component: 56, sample: 1)>
 array([[[-1.10796361e-02],
         [-9.50037320e-03],
         [ 2.30356729e-04],
         [ 6.61311350e-03],
         [-2.89780038e-03],
         [ 8.36996771e-03],
         [ 2.65897298e-02],
         [ 2.12272067e-03],
         [-1.95695310e-03],
         [-1.35591253e-02],
         [-6.76253951e-03],
         [-6.68418220e-03],
         [ 8.65900826e-03],
         [ 8.06463974e-03],
         [ 8.75432191e-03],
         [ 1.79226446e-03],
         [-1.23357701e-02],
         [-9.71353308e-03],
         [-5.83904413e-03],
         [-3.76564045e-03],
 ...
         [ 2.48001636e-01],
         [ 3.19232221e-01],
         [ 3.35568842e-01],
         [ 5.22499048e-01],
         [ 5.25357151e-01],
         [ 1.99091820e-01],
         [ 5.70233193e-01],
         [ 4.60076470e-02],
         [ 6.61518754e-01],
         [ 5.28141459e-01],
         [ 3.26251290e-01],
         [ 2.60457422e-01],
         [ 5.49458118e-01],
         

In [60]:
y_pred = model.predict(n=1)
scaler.inverse_transform(y_pred)['hosp']

<TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
array([[[1789.50577694]]])
Coordinates:
  * time       (time) datetime64[ns] 2021-04-22
  * component  (component) object 'hosp'
Dimensions without coordinates: sample